In [1]:
import pandas as pd
import numpy as np
import torch


In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [3]:
# trainX = train_df.iloc[:,:-1]
# trainY = train_df.iloc[:,-1:]
# testX = test_df.iloc[:,:-1]
# testY = test_df.iloc[:,-1:]

In [4]:
# trainx_torch = torch.from_numpy(trainX.values.astype(np.float64))
# trainy_torch = torch.from_numpy(trainY.values.astype(np.float64))
# testx_torch = torch.from_numpy(testX.values.astype(np.float64))
# testy_torch = torch.from_numpy(testY.values.astype(np.float64))
trainx_torch = torch.tensor(train_df.values[:, :-1], dtype=torch.float64)
trainy_torch = torch.tensor(train_df.values[:, -1:], dtype=torch.float64)
testx_torch = torch.tensor(test_df.values[:, :-1], dtype=torch.float64)
testy_torch = torch.tensor(test_df.values[:, -1:], dtype=torch.float64)

In [5]:
#q1
trainx_torch.nelement() * trainx_torch.element_size()

83520

In [6]:
trainx_torch_16 = trainx_torch.to(torch.float16)
trainX_torch_64 = trainx_torch_16.to(torch.float64)
max_diff_torch = torch.max(torch.abs(trainX_torch_64  - trainx_torch))

In [7]:
#q2
max_diff_torch.item()

0.0

In [8]:
#q3
torch.cuda.is_available()

False

In [9]:
#q4
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=trainx_torch.dtype)
(testx_torch[0,:] @ coef).item()

9.844

In [10]:
#q5
torch.mean(testx_torch @ coef).item()

12.073632183908048

In [11]:
#q6
def func(x):
    return x*x - 8*x + 19
x = torch.tensor(0.0)
y = func(x)
float(y)

19.0

In [12]:
#q7
x = torch.tensor(0.0, requires_grad=True)
opt = torch.optim.SGD([x], lr=0.1)
for epoch in range(100):
    # print("Iteration: ", epoch)
    y = func(x)
    # plt.plot(x.detach(), y.detach(), "ro", markersize=3+epoch)
    y.backward()
    # print(x.grad)
    opt.step()
    opt.zero_grad()
x.item()

3.999999523162842

In [13]:
#q8
ds = torch.utils.data.TensorDataset(trainx_torch, trainy_torch)
dl = torch.utils.data.DataLoader(ds, batch_size=50, shuffle=True)
loss_fn = torch.nn.MSELoss()
coef = torch.zeros((trainx_torch.shape[1], trainy_torch.shape[1]), dtype=torch.float64, requires_grad=True)
loss_fn(trainx_torch @ coef, trainy_torch).item()

197.8007662835249

In [14]:
loss_fn = torch.nn.MSELoss()
torch.manual_seed(544)
#coef = torch.rand((trainx_torch.shape[1], trainy_torch.shape[1]), dtype=torch.float64, requires_grad=True)
coef = torch.zeros((trainx_torch.shape[1], trainy_torch.shape[1]), dtype=torch.float64, requires_grad=True)

In [15]:
#q9
optimizer = torch.optim.SGD([coef], lr=0.000002)
for epoch in range(500):
    for batchx, batchy in dl:
        predictions = batchx @ coef
        loss = loss_fn(predictions, batchy)
        loss.backward()  #computes coef.grad
        optimizer.step()
        optimizer.zero_grad()
    # print(loss)
loss_fn(trainx_torch @ coef, trainy_torch).item()

26.8113940147193

In [16]:
#q10
loss_fn(testx_torch @ coef, testy_torch).item()

29.05854692548551